In [63]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolAlign

In [64]:
n_mol = 3
writefile = 'lxr' + str(n_mol) + '.sdf'
#mols = Chem.SDMolSupplier('ICG_001_analog.sdf')
mols = Chem.SDMolSupplier('lxr.sdf')
ms = [Chem.AddHs(m) for m in mols]

In [65]:
m = ms[n_mol]
Chem.FindMolChiralCenters(m,includeUnassigned=True)

[(6, '?'), (7, '?'), (8, '?'), (10, '?'), (15, '?')]

In [66]:
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions
isomers = tuple(EnumerateStereoisomers(m))

In [62]:
Chem.AssignAtomChiralTagsFromStructure(m) # this is important for 3D structure
print(Chem.FindMolChiralCenters(m))

[(6, 'R'), (7, 'R'), (8, 'R'), (10, 'R'), (15, 'S')]


In [54]:
ps = AllChem.ETKDGv3()
ps.randomSeed=0xf00d
ps.pruneRmsThresh = 0.5
#m = ms[n_mol]
Chem.AssignAtomChiralTagsFromStructure(m) # this is important for 3D structure
print(Chem.FindMolChiralCenters(m))
cids = AllChem.EmbedMultipleConfs(m,50,ps)

[(6, 'R'), (7, 'R'), (8, 'R'), (10, 'R'), (15, 'S')]


In [55]:
mp = AllChem.MMFFGetMoleculeProperties(m, mmffVariant='MMFF94s')
AllChem.MMFFOptimizeMoleculeConfs(m, numThreads=0,mmffVariant='MMFF94s')


[(1, 87.17135243848486),
 (1, 82.08862019438158),
 (1, 84.31784356001765),
 (1, 87.17134280309732),
 (0, 80.17254923336486),
 (0, 83.38986919261627),
 (0, 87.10858811021639),
 (0, 79.47495200002527),
 (0, 82.08861994285981),
 (0, 81.53007353744542),
 (0, 82.04110217015591),
 (0, 83.84700961527524),
 (0, 87.10858792796276),
 (0, 78.21384759395191),
 (1, 86.58888802522816),
 (1, 82.0886198824204),
 (1, 83.56391701406278),
 (1, 85.25302696364454),
 (0, 90.20577333730469),
 (0, 84.7193490678262),
 (0, 81.0085590051565),
 (0, 82.72628030191187),
 (0, 86.08385773165773),
 (1, 90.10562231292714),
 (0, 81.62971811076127)]

In [56]:
w = Chem.SDWriter(writefile)
res = []
for cid in cids:
        ff = AllChem.MMFFGetMoleculeForceField(m, mp, confId=cid)
        e = ff.CalcEnergy()
        res.append((cid, e))

sorted_res = sorted(res, key=lambda x:x[1])
rdMolAlign.AlignMolConformers(m)

for cid, e in sorted_res:
        m.SetProp('CID', str(cid))
        m.SetProp('Energy', str(e))
        w.write(m, confId=cid)
w.close()